In [398]:
# Imports
import numpy as np
import random
import math
# import networkx as nx
# from scipy.spatial.distance import pdist, squareform
# import matplotlib.pyplot as plt

In [399]:
# Function and CLass definitions
class Print_order():
    def __init__(self, poster_order, fitness = 0):
        self.poster_order = poster_order
        self.fitness = fitness

class Poster(): # stores details of a poster
    def __init__(self, ink_total, inks):
        
        #ink_total referring to total number of inks used for this poster
        self.ink_total = ink_total
        # inks referring a list of inks used for this poster [ints]
        self.inks = inks
        
        #TODO: 
    
class Print_instance(): # i.e. unit of print time (can consist of multiple posters)
    def __init__(self, inks, poster_count = 1):
        # inks referring to number a list of inks used in the print instance
        self.final_inks = inks
        # posterCount referring to the number of posters to be printed in this print instance.
        self.poster_count = poster_count
        
        #TODO: D
    
def get_orders_details(filename):
    """
    Getting and returning order_list from input
    :returns: orders_list, N = Number of inks, T = Time constraint
    """
    
    f = open(filename)
    
    # Header Handling
    header_list = f.readline().split(" ")
    N = header_list[0]
    T = header_list[2]
    
    # Orders Handling 
    orders_list = []
    for line in (l[:-1] for l in f):
        order_line = line.split(" ")
        ink_total = order_line.pop(0)
        inks = order_line
        poster = Poster(ink_total, inks)
        
        orders_list.append(poster)
        
    f.close()
    
    return orders_list, int(N), int(T)
    # each poster in list should be of object type Poster
    # TODO: get orders list input
    
def elitism_selection(sorted_population, max_pop_size, survivor_count):
    """
    "Keep best half" selection approach
    """
    
    elitism_population = []   

    for i in range(max_pop_size - survivor_count):
        elitism_population.append(sorted_population[i])
            
    return elitism_population

# TODO: consider different crossover techniques(?), e.g. N-point crossover
def single_point_crossover(parent1, parent2):
    crossover_index = random.randint(0, len(parent1))
    
    # 1st parent
    p1_left = parent1[0:crossover_index]
    p1_right = parent1[crossover_index:(len(parent1) - 1)]
    # 2nd parent
    p2_left = parent2[0:crossover_index]
    p2_right = parent2[crossover_index:(len(parent1) - 1)]
    
    # Root
    child1 = p1_left
    child2 = p2_left
    
    # Child 1
    for val in p2_right:
        if val not in child1:
            np.append(child1, val)
        else: 
            for val in p2_left:
                if val not in child1:
                    np.append(child1, val)
    
    # Child 2
    for val in p2_right:
        if val not in child1:
            np.append(child2, val)
        else: 
            for val in p2_left:
                if val not in child1:
                    np.append(child2, val)
                
    return child1, child2
    
    # TODO: return 2 children (crossed over, likely?)
    
def mutate(child, mutation_rate):
    
    rand = random.random()
    if rand < mutation_rate:
        swap_index1 = random.randint(0, len(child))
        swap_index2 = random.randint(0, len(child))
        
        # Ensuring swap index are not the same
        while swap_index1 != swap_index2:
            swap_index2 = random.randint(0, len(child))
        
        # Swapping
        temp = child[swap_index1]
        child[swap_index1] = child[swap_index2]
        child[swap_index2] = temp
        
    return child
    
def tetrisify(candidate):
    """[summary]
    :param candidate:
    :returns: List of Print_instance objects
    """
    
    tetrified_candidate = []
    # TODO: candidate begins as Print_instance list
    # Converting Poster list to Print_instance list
    for poster in candidate:
        tetrified_candidate.append(Print_instance(poster.inks))
    
    # Checking for duplication of inks 
    # len(candidate)- 1 so that last poster does not check
    for i in range(len(tetrified_candidate) - 1):
        if i+1 < len(tetrified_candidate):
            can_compress = True
            for ink in tetrified_candidate[i].final_inks:
                # For ink in poster...
                if ink in tetrified_candidate[i + 1].final_inks:
                    # If ink in next poster's list of inks
                    can_compress = False
                    # break

            
            # Creating combined or individual Print_instance objects to append
            # to tetrisified_candidate list.
            if can_compress:
                # If no duplicated inks...
                combined_inks = tetrified_candidate[i].final_inks + tetrified_candidate[i + 1].final_inks
                #print_instance = Print_instance(combined_inks, 2)
                tetrified_candidate[i].final_inks = combined_inks
                tetrified_candidate[i].poster_count += 1
                
                # Deleting 2nd print_instance, as it is combined
                del tetrified_candidate[i + 1]
                i -= 1
    
    return tetrified_candidate

def fitness(candidate, T, N, success_income = 10, fail_cost = 5):
    """
    Calculating a fitness score of a candidate solution, given the T constraint
    """
    tetrisified_candidate = tetrisify(candidate)
    
    income = 0
    loss = 0
    
    # For each Print_instance unit of time, until time constraint T... 
    for i in range(T):
        poster_count = tetrisified_candidate[i].poster_count
        
        if len(tetrisified_candidate[i].final_inks) < N:
            income += poster_count * success_income
            
        else:
            loss += poster_count * fail_cost
            
    return income - loss
    
    
def sort_by_fitness(population, T, N):
    # TODO: Sort function using fitness function
    
    print_order_population = []
    for ind in population:
        print_order = Print_order(ind, fitness(ind, T, N,))
        print_order_population.append(print_order)
        
    # Sorting population using list of Print_Order objects (to more easily store fitness)
    print_order_population.sort(key=lambda x: x.fitness, reverse=True)
    
    sorted_population = []
    for print_order in print_order_population:
        sorted_population.append(print_order.poster_order)
    
    return sorted_population

def get_next_generation(sorted_population, max_pop_size, survivor_count, mutation_rate):    
    # TODO: not rely on this
    # Using elitism_selection (best half) approach
    
    # TODO: implement other selection methods
    survivor_population = elitism_selection(sorted_population, max_pop_size, survivor_count)
    children = []
    
    # If survivor_population is odd, then duplicate the best fitting candidate and add to front of survivor population
    if len(survivor_population) / 2 % 2 == 1:
        survivor_population.insert(0, survivor_population[0])
    
    # Crossover
    for i in range(len(survivor_population)):
        # Getting parents
        if i > len(survivor_population) + 1:
            p1 = survivor_population[i]
            p2 = survivor_population[i+1]
        
            # Crossing over children using parents
            child1, child2 = single_point_crossover(p1, p2)
            
            # Mutating children
            for i in range(len(children)):
                mutated1 = mutate(child1, mutation_rate)
                mutated2 = mutate(child2, mutation_rate)
                # Appending to children list for next population
                children.append(mutated1)
                children.append(mutated2)
            
        i += 1       
    
    
    # Trimming returned population such that it fits max_pop_size
    return_population = survivor_population + children
    return_population[:max_pop_size]
    return return_population
    
    
def genetic_algorithm(orders, T, N, max_pop_size = 20, max_generations = 100, survivor_count = 10, mutation_rate = 0.1):
    """
    [DESC ALAJKSDLASKJDASLKDJASLDKJASLDKAJSDLAKSJDLAKSJDALSKDJASLDKJASLKDJASD]
    :param orders: 
    :param T: Time constraint
    :param N: Total Ink count (2 <= N <= 32) -- Consider odd/even calculations
    :param max_pop_size:
    :param max_generations:
    """
    
    # Defining initial population with random permutations
    population = [] 
    for i in range (max_pop_size):
        # TODO: check this permutation implementation
        printOrders = np.random.permutation(orders)
        population.append(printOrders)
        
    # Performance-tracking Variables
    optimal_solution_found = None
    # TODO: convert solution to solution print order(below)
    optimal_print_order = []
    optimal_fitness_score = -np.inf
    generation_bests = []
    
    # Generation loop
    for i in range(max_generations):
        # Sorting population by fitness
        sorted_population = sort_by_fitness(population, T, N,)
        
        # TODO: consider population average for analysis (graph representation)
        # Comparing population optimal 
        population_best = fitness(sorted_population[0], T, N) #TODO: total print costs
        if population_best > optimal_fitness_score: # Maximisation 
            optimal_fitness_score = population_best
            optimal_solution_found = sorted_population[0]
        generation_bests.append(population_best)
        
        # TODO: consider here an option for selecting selection method(s), for critical evaluation
        population = get_next_generation(sorted_population, max_pop_size, survivor_count, mutation_rate)        
        
    return optimal_solution_found, optimal_fitness_score, generation_bests
    
    

    

In [400]:
# Main Function
orders_list, N, T  = get_orders_details("orders.txt")

optimal_solution_found, optimal_fitness_score, generation_bests = genetic_algorithm(orders_list, T, N)
solution_string = ""
for poster in optimal_solution_found:
    for ink in poster.inks:
        solution_string += str(ink) + " "
    solution_string += "\n"
print(f"Best Solution:\n{solution_string}. Fitness Score: {optimal_fitness_score}")
print(f"Generation Bests:\n {generation_bests}")

Best Solution:
1 2 4 

0 2 3 
1 5 
0 1 3 5 
4 
. Fitness Score: 60
Generation Bests:
 [60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60]


In [401]:
# Plotting Results